In [1]:
!pwd

/Users/guilherme.yambanis/Documents/CovidMDP/notebooks


In [2]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/guilherme.yambanis/Documents/CovidMDP/src"))

In [7]:
os.chdir("../src/models")

In [8]:
from numpy.random import default_rng
import numpy as np
from tqdm import tqdm

In [9]:
from models.simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population

In [10]:
from actions import city_restrictions

In [11]:
policy='Unrestricted'
days=350
step_size=7
disable_tqdm=False
seed=None

In [12]:
prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

g_pickle = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [13]:
rng = default_rng(seed)
pop_matrix, adj_list = init_infection(g_pickle)
data = []
total_steps = int(np.ceil(days/step_size))

if isinstance(policy, str):
    policy = total_steps * [policy]

if len(policy) < total_steps:
    raise ValueError(f'len of policy should be at least {total_steps}')

for day in tqdm(range(1, days+1), disable=disable_tqdm):
    # if more than 90% already recovered, break simulation
    if (pop_matrix[pop_matrix[:, 1] == -1].shape[0] > pop_matrix.shape[0]*.9):
        break

    if day % step_size == 1:          
        current_step = int(day/step_size)
        restrictions = city_restrictions[policy[current_step]]

    pop_matrix = spread_infection(pop_matrix, adj_list, restrictions, day, rng, p_r)
    pop_matrix = lambda_leak_expose(pop_matrix, day)
    pop_matrix = update_population(pop_matrix)

    data.append(pop_matrix[:, 0:2])

100%|█████████████████████████████████████████| 350/350 [00:06<00:00, 53.61it/s]


In [14]:
import pandas as pd

In [15]:
pd.DataFrame([pd.Series(d[:, 1]).value_counts() for d in data])

,-1,0,1,2,3
0,NaN,55460.0,32.0,NaN,NaN
1,NaN,55458.0,33.0,1.0,NaN
2,NaN,55455.0,31.0,6.0,NaN
3,NaN,55453.0,27.0,12.0,NaN
4,1.0,55447.0,29.0,15.0,NaN
...,...,...,...,...,...
345,41167.0,14271.0,16.0,29.0,9.0
346,41177.0,14269.0,17.0,22.0,7.0
347,41180.0,14267.0,14.0,24.0,7.0
348,41184.0,14263.0,15.0,23.0,7.0


In [16]:
import networkx as nx

In [17]:
for x in p_r.items():
    print(x)

('home', 0.06)
('neighbor', 0.006)
('work', 0.006)
('school', 0.009)


In [18]:
G = nx.read_gpickle(g_pickle)


In [19]:
unique_types = np.unique([data['edge_type'] for x,y, data in G.edges(data=True)])

edge_list = {
    edge_type: np.array([(u,v) for u,v,e in G.edges(data=True) if e['edge_type'] == edge_type])
                            for edge_type in unique_types
}

In [58]:
currently_infected = rng.choice(G.nodes, size=5000)

In [59]:
%%timeit
exposed = []

def filter_contacts(rel, prob, restrictions, currently_infected, edge_list):
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])

    return contacts[mask]

args = [restrictions, currently_infected, edge_list]
exposed = list(map(lambda x: filter_contacts(x[0], x[1], *args), [x for x in p_r.items()]))
exposed = np.unique(np.concatenate(exposed)) 

46.4 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
%%timeit

exposed = []
for rel, prob in p_r.items():
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])
    exposed += [contacts[mask]]

exposed = np.unique(np.concatenate(exposed))

48.2 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%%timeit

def get_exposed(p_r, currently_infected, restrictions):
    exposed = []
    for relation, prob in p_r.items():
        pairs = [
            contact 
            for i in currently_infected
            for contact in adj_list[relation][i]     
        ]
        pairs = np.array(pairs)
        chances = rng.random(size=len(pairs)) < prob * (1-restrictions[relation])

        exposed.extend(pairs[chances])

    exposed = np.unique(exposed).astype(int)

    return exposed

exposed = get_exposed(p_r, currently_infected, restrictions)

26.7 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
%%timeit

def get_exposed_numpy(p_r, currently_infected, restrictions):
    exposed = []
    for relation, prob in p_r.items():
        pairs = np.concatenate(list(map(adj_list[relation].get, currently_infected))).flatten()
        chances = rng.random(size=len(pairs)) < prob * (1-restrictions[relation])

        exposed.extend(pairs[chances])

    exposed = np.unique(exposed).astype(int)

    return exposed

exposed = get_exposed(p_r, currently_infected, restrictions)

27.9 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
